[twitter api reference index](https://developer.twitter.com/en/docs/api-reference-index)


# POST oauth2/token
Allows a registered application to obtain an OAuth 2 Bearer Token, which can be used to make API requests on an application's own behalf, without a user context.
https://developer.twitter.com/en/docs/basics/authentication/api-reference/token


Once you have create app, click on app details and keys and token tab

<font size="6" color='red'> write API key value for client_key and API secret key value for client_secret in the next cell</font>

In [98]:
import requests
import base64
from copy import deepcopy
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from textblob import TextBlob
from bs4 import BeautifulSoup as bsoup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import nltk
import re
import ast
import emoji

client_key = "uDxiPfIPIcFjGD0fmxj7c2XAk"
client_secret = "Bcr55JD2dNhGFftvtWwn19x78pRPLNP5rCBmGw0yVeW5Uy3hpc" 

In [12]:
key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

In [13]:
base_url = 'https://api.twitter.com/'
auth_endpoint = base_url+'oauth2/token'

auth_headers = { 'Authorization': 'Basic {}'.format(b64_encoded_key),
                'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}

auth_data = { 'grant_type': 'client_credentials'}



Let's post to authentication endpoint and get the token to be used in subsequent REST API call

In [14]:
response = requests.post(auth_endpoint, headers=auth_headers, data=auth_data)
response.status_code

200

In [15]:
json_data =  response.json()
json_data

{'token_type': 'bearer',
 'access_token': 'AAAAAAAAAAAAAAAAAAAAAEskIQEAAAAAqkugIbkkPuB3XUUBhHAYkbYX4ik%3DjxVjLVRb5O9OUZ95hy8Z3cDHaTkA4j7VJM0Shx3GNLq2W8SdOA'}

In [16]:
access_token = json_data['access_token']

# Let's do some search on climate change
This is how we got search endpoint

https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

In [17]:
max_tweets=1000
tw_limit=100# we can get only 100 tweet per call with standard search api
tweets = []

In [18]:
search_headers = {'Authorization': 'Bearer {}'.format(access_token)    
}

keywords = ["Packers","Bears","Eagles", "Cowboys","Raiders", "ChiefsKingdom"]

for keyword in keywords:

    parameters = { 'q': keyword,
                        'result_type': 'recent',
                    'count': tw_limit }

    search_url = base_url+'1.1/search/tweets.json'

    response = requests.get(search_url, headers=search_headers, params=parameters)
    football_tweets = response.json()

    for status in football_tweets['statuses']:
        status["team"] = keyword
    tweets += football_tweets['statuses']
    ids = [tw['id'] for tw in football_tweets['statuses']]
    min_id = min(ids)
    tw_ids = []
    tw_ids+=ids

    for  i in range(max_tweets//tw_limit -1):
        parameters = { 'q': keyword,
                        'result_type': 'recent',
                    'count': tw_limit,
                    'max_id': min(tw_ids)-1
                     }
        print("searching tweets with id  < {}".format(min_id))
        search_url = base_url+'1.1/search/tweets.json'
        response = requests.get(search_url, headers=search_headers, params=parameters)
        football_tweets = response.json()
        ids = [tw['id'] for tw in football_tweets['statuses']]
        tw_ids+=ids
        print(i, len(football_tweets['statuses']))
        min_id = min(ids)
        for status in football_tweets['statuses']:
            status["team"] = keyword
        tweets += football_tweets['statuses']    

searching tweets with id  < 1326754119294611456
0 100
searching tweets with id  < 1326744311950696453
1 100
searching tweets with id  < 1326738960668823552
2 100
searching tweets with id  < 1326733895413403648
3 100
searching tweets with id  < 1326729113693024256
4 100
searching tweets with id  < 1326724205019344897
5 100
searching tweets with id  < 1326720600124043264
6 100
searching tweets with id  < 1326718311363735552
7 100
searching tweets with id  < 1326712999231578118
8 100
searching tweets with id  < 1326762526609186816
0 100
searching tweets with id  < 1326760244870000649
1 100
searching tweets with id  < 1326757359495024640
2 100
searching tweets with id  < 1326755439657308160
3 100
searching tweets with id  < 1326753084899520512
4 100
searching tweets with id  < 1326751406871760896
5 100
searching tweets with id  < 1326749270871928833
6 100
searching tweets with id  < 1326747445112557568
7 100
searching tweets with id  < 1326745379442995200
8 91
searching tweets with id  < 1

As per twitter api documentation we can use

max_id:Returns results with an ID less than (that is, older than) or equal to the specified ID.

to get older tweets

# Q1 (.5 points) Modify only *parameters* dictionary in the following code so that subsequent call to climate change search returns older tweets with id less than minimum of previous tweets ids.

In [27]:
print('Total tweets retrieved= {}'.format(len(tweets)))

Total tweets retrieved= 5991


# Let's print some tweets

In [28]:
import pprint #pretty-print” arbitrary Python data structures. Tweets object has hierarchical struture
max_tweet_to_print=4

In [29]:
pprint.pprint(tweets[0:max_tweet_to_print])

bg.png',
                             'profile_background_tile': False,
                             'profile_banner_url': 'https://pbs.twimg.com/profile_banners/256590867/1604871893',
                             'profile_image_url': 'http://pbs.twimg.com/profile_images/1320365218757828610/LoHXR6fd_normal.jpg',
                             'profile_image_url_https': 'https://pbs.twimg.com/profile_images/1320365218757828610/LoHXR6fd_normal.jpg',
                             'profile_link_color': '981CEB',
                             'profile_sidebar_border_color': '000000',
                             'profile_sidebar_fill_color': '000000',
                             'profile_text_color': '000000',
                             'profile_use_background_image': False,
                             'protected': False,
                             'screen_name': 'OfficialShegs',
                             'statuses_count': 163293,
                             'time_zone': None,
       

Link to understand [tweet-object](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object)

Let'see some tweets

In [30]:
for status in tweets[0:max_tweet_to_print]:
    print(status['text'] + '\n')

@pelicansfan13 @AdamSchefter Unlike packers and seahawks...we aint gotta worry bout yall old ass qb throwin deep. T… https://t.co/kxSJoI100t

RT @sfl53: Week 2 game has both SFL Rams and SFL Packers searching for a defense.  The two teams battled to a 52-48 Packers win in Los Ange…

Greenbay Packers all day !!!! #GoPackGo https://t.co/h76pz60433

RT @packers: 3️⃣3️⃣ 

@Showtyme_33 | #GoPackGo https://t.co/mkkHsjvFgJ



# Pandas DataFrame for storing tweet feed

Create following date frame for storing tweets

- **climate_feed_df** DataFrame for storing tweet
- **user_df** DataFrame for storing user detail
- **retweeted_status_df** for storing original tweet

# Printing keys in tweets

In [31]:
print(tweets[1].keys())
print(len(tweets[1].keys()))


dict_keys(['created_at', 'id', 'id_str', 'text', 'truncated', 'entities', 'metadata', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'retweeted_status', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang', 'team'])
26


In [32]:
import pandas as pd

# Generators for building DataFrames

In [33]:
def tweet_generator(tweets):
    for tweet in tweets:
        modified_tweet= {}
        
        #Write your code here

        modified_tweet = deepcopy(tweet)
        try:
            modified_tweet["user_id"] = modified_tweet["user"]["id"]
            modified_tweet.pop("user", None)
        except KeyError:
            pass
        try:
            modified_tweet["retweeted_status_id"] = modified_tweet["retweeted_status"]["id"] 
            modified_tweet.pop("retweeted_status", None)
        except KeyError:
            pass
        yield modified_tweet       
            
            

In [34]:
# Checking  generator output
list(tweet_generator(tweets[0:2]))

[{'created_at': 'Thu Nov 12 05:54:09 +0000 2020',
  'id': 1326765205024632835,
  'id_str': '1326765205024632835',
  'text': '@pelicansfan13 @AdamSchefter Unlike packers and seahawks...we aint gotta worry bout yall old ass qb throwin deep. T… https://t.co/kxSJoI100t',
  'truncated': True,
  'entities': {'hashtags': [],
   'symbols': [],
   'user_mentions': [{'screen_name': 'pelicansfan13',
     'name': 'Nick Traylor',
     'id': 1245850911857139712,
     'id_str': '1245850911857139712',
     'indices': [0, 14]},
    {'screen_name': 'AdamSchefter',
     'name': 'Adam Schefter',
     'id': 51263592,
     'id_str': '51263592',
     'indices': [15, 28]}],
   'urls': [{'url': 'https://t.co/kxSJoI100t',
     'expanded_url': 'https://twitter.com/i/web/status/1326765205024632835',
     'display_url': 'twitter.com/i/web/status/1…',
     'indices': [117, 140]}]},
  'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
  'source': '<a href="http://twitter.com/download/android" rel="nof

In [35]:
football_feed_df = pd.DataFrame(tweet_generator(tweets))

# Text Cleanup

Here, we're going to take to start cleaning the text in the tweets.

In [51]:
print("Rows with either null text of teams: {}".format(football_feed_df[["text","team"]].isnull().any(axis = 1).sum())) 

Rows with either null text of teams: 0


In [49]:
def NoneToNan(elem):
    if elem == "None" or elem == None:
        return np.nan
    else:
        return elem

football_feed_df = football_feed_df.applymap(NoneToNan)

In [52]:
football_feed_df[["text","team"]]

,text,team
0,@pelicansfan13 @AdamSchefter Unlike packers an...,Packers
1,RT @sfl53: Week 2 game has both SFL Rams and S...,Packers
2,Greenbay Packers all day !!!! #GoPackGo https:...,Packers
3,RT @packers: 3️⃣3️⃣ \n\n@Showtyme_33 | #GoPack...,Packers
4,RT @fbgchase: Aaron Rodgers has started 182 ga...,Packers
...,...,...
5986,Gonna be a fun one! #MVP #MahomesMadness #Chie...,ChiefsKingdom
5987,RT @ArrowheadLive: Have you lost passion for f...,ChiefsKingdom
5988,RT @PFF_Chiefs: #Chiefs TE Travis Kelce’s 2020...,ChiefsKingdom
5989,RT @PFF_Chiefs: #Chiefs TE Travis Kelce’s 2020...,ChiefsKingdom


In [58]:
hashtags = []
user_mentions = []

for row in football_feed_df.itertuples():
    row = row.entities
    try:
        hashtags_in_row = row["hashtags"]
        hashtags.append(hashtags_in_row)
    except:
        hashtags.append('')
    try:
        user_mentions_in_row = row["user_mentions"][0]["name"]
        user_mentions.append(user_mentions_in_row)
    except:
        user_mentions.append('')



In [59]:
football_feed_df["hashtags"] = hashtags
football_feed_df["user_mentions"] = user_mentions
football_feed_df.head()

,created_at,id,id_str,text,truncated,entities,metadata,source,in_reply_to_status_id,in_reply_to_status_id_str,...,team,user_id,retweeted_status_id,quoted_status_id,quoted_status_id_str,quoted_status,possibly_sensitive,extended_entities,hashtags,user_mentions
0,Thu Nov 12 05:54:09 +0000 2020,1326765205024632835,1326765205024632835,@pelicansfan13 @AdamSchefter Unlike packers an...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",1.326643e+18,1326643202640801795,...,Packers,1018431728,NaN,NaN,NaN,NaN,NaN,NaN,[],Nick Traylor
1,Thu Nov 12 05:53:46 +0000 2020,1326765109734203395,1326765109734203395,RT @sfl53: Week 2 game has both SFL Rams and S...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,NaN,...,Packers,1157637917038264320,1.326761e+18,NaN,NaN,NaN,NaN,NaN,[],sfl53
2,Thu Nov 12 05:53:38 +0000 2020,1326765073302482945,1326765073302482945,Greenbay Packers all day !!!! #GoPackGo https:...,False,"{'hashtags': [{'text': 'GoPackGo', 'indices': ...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,NaN,...,Packers,28899488,NaN,1.320375e+18,1320375296604909569,{'created_at': 'Sun Oct 25 14:42:56 +0000 2020...,False,NaN,"[{'text': 'GoPackGo', 'indices': [30, 39]}]",
3,Thu Nov 12 05:53:33 +0000 2020,1326765052339384324,1326765052339384324,RT @packers: 3️⃣3️⃣ \n\n@Showtyme_33 | #GoPack...,False,"{'hashtags': [{'text': 'GoPackGo', 'indices': ...","{'iso_language_code': 'und', 'result_type': 'r...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,...,Packers,1069614443875717120,1.326737e+18,NaN,NaN,NaN,False,"{'media': [{'id': 1326736684839546880, 'id_str...","[{'text': 'GoPackGo', 'indices': [37, 46]}]",Green Bay Packers
4,Thu Nov 12 05:53:11 +0000 2020,1326764959771013120,1326764959771013120,RT @fbgchase: Aaron Rodgers has started 182 ga...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,...,Packers,846892986021527552,1.326523e+18,NaN,NaN,NaN,NaN,NaN,[],Football Perspective


In [60]:
# write code here

def clean_tweet(text):  

    url = re.compile('https://.+')
    rt = re.compile("RT")
    user = re.compile('@\S+\s')
 
    #'(RT|@.*\s|https://.+)'

    text = url.sub("", text)
    text = rt.sub("", text)
    text = user.sub("", text)

    return text

clean_tweet("RT .@SteveKingIA says he's unsure about the science of climate change. But you don’t need to be a scientist to take ac… https://t.co/J1CcrQZkB2")

football_feed_df["text"] = football_feed_df["text"].apply(clean_tweet)

In [67]:
# write code here
def emoji_retriever(text):
      return [(char, emoji.demojize(char)) for char in text if char in emoji.UNICODE_EMOJI]

football_feed_df["emoji"] = football_feed_df["text"].apply(emoji_retriever)

In [90]:
contraction_patterns = [(r'won\'t', 'will not'), (r'can\'t', 'cannot'),
(r'haven\'t', 'have not'),
(r'(\w+)\'ll', '\g<1> will'),
(r'(\w+)\'re', '\g<1> are')]

class contraction_replacer(object):
    def __init__(self, contraction_patterns):
        self._contraction_regexes = [(re.compile(p), replaced_text) for p, replaced_text in contraction_patterns]

    def do_contraction_normalization(self, text):
        for contraction_regex, replaced_text in self._contraction_regexes:
            text = contraction_regex.sub(replaced_text, text)
        return text

def preprocess_text(text):
    
    contraction_patterns = [(r'won\'t', 'will not'), (r'can\'t', 'cannot'),
    (r'haven\'t', 'have not'),
    (r'(\w+)\'ll', '\g<1> will'),
    (r'(\w+)\'re', '\g<1> are')]

    replacer = contraction_replacer(contraction_patterns)
    text = replacer.do_contraction_normalization(text)
    text = nltk.tokenize.word_tokenize(text)

    stop_words = set(stopwords.words('english')) 
    
    # lemmatization
    wnlemma = WordNetLemmatizer()
    lemma_text = []
    for word in text:
        if word in stop_words:
            continue
        lemma_text.append(wnlemma.lemmatize(word))
    lemma_text = " ".join(lemma_text)

    return lemma_text

football_feed_df["processed_text"] = football_feed_df["text"].apply(preprocess_text)

In [121]:
def sentiment_analysis(text):
    text = " ".join(text["processed_text"])
    text = TextBlob(text)

    return text.sentiment.polarity

teams_df = football_feed_df[["processed_text", "team"]].groupby("team")
sentiments = teams_df.apply(sentiment_analysis)

# Sentiment Analysis

Below represents the relative polarity of each team from each team on Twitter.

In [122]:
sentiments

team
Bears            0.094130
ChiefsKingdom    0.072478
Cowboys          0.228591
Eagles           0.148519
Packers          0.123480
Raiders          0.224921
dtype: float64

In [43]:
def tweet_user_generator(tweets):
    for tweet in tweets:
        yield tweet["user"]
                 

In [28]:
user_df = pd.DataFrame(tweet_user_generator(tweets))

In [29]:
user_df.head()

,id,id_str,name,screen_name,location,description,url,entities,protected,followers_count,...,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,has_extended_profile,default_profile,default_profile_image,following,follow_request_sent,notifications,translator_type
0,1310711812799766529,1310711812799766529,BringerofPeace,BringerofP,,Good vibez | other media 📷 Frankobangg | 👻 Fra...,None,{'description': {'urls': []}},False,12,...,DDEEF6,333333,True,True,True,False,None,None,None,none
1,874131570,874131570,Bob Strulavitch,Bobstrulavitch,,,None,{'description': {'urls': []}},False,214,...,000000,000000,False,False,False,False,None,None,None,none
2,203445197,203445197,Nate L,houseofannie,"ÜT: 37.2643,-121.971226",House of Annie is the food and travel blog of ...,http://t.co/WdcBUY2zoQ,{'url': {'urls': [{'url': 'http://t.co/WdcBUY2...,False,2803,...,FFFFFF,333333,True,False,False,False,None,None,None,none
3,1701716760,1701716760,Tom Achee,Tilsder_,"Sandy Springs, GA",Recent graduate from Full Sail University who'...,None,{'description': {'urls': []}},False,59,...,000000,000000,False,False,False,False,None,None,None,none
4,1073024519114125312,1073024519114125312,🛋️,beej310,,https://t.co/LoT4i0QjfM,None,{'description': {'urls': [{'url': 'https://t.c...,False,17,...,DDEEF6,333333,True,False,True,False,None,None,None,none
